[![Binder](https://mybinder.org/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia-13/main?filepath=03%20Use%20Python%20with%20PyCall%20and%20PythonCall.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>

# Fall-in-love-with-Julia: Accelerating Python 101

For Julia Python interactions there are two packages:

|    | [PyCall.jl](https://github.com/JuliaPy/PyCall.jl) | [PythonCall.jl](https://github.com/cjdoris/PythonCall.jl) |
| -: | --------- | ------------- |
| pypi | with [`PyJulia`](https://github.com/JuliaPy/pyjulia) python package (simply called `julia` on pypi) | with [`JuliaCall`](https://pypi.org/project/juliacall/) python package |
| conversions | automatically converts between native types | no auto-conversion, just wrapping |
| dependencies | Global package management via `Conda.jl` | Project-separated package management via `CondaPkg.jl` |
| run python | use `py"..."` | use `@pyexec "..."` |

Outline
- [01](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia-13/main?filepath=01%20Accelerating%20Python%20with%20PyJulia.ipynb) first notebook shows how to use PyJulia
- [02](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia-13/main?filepath=02%20Accelerating%20Python%20with%20JuliaCall.ipynb) second notebook is about JuliaCall
- **[03](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia-13/main?filepath=03%20Use%20Python%20with%20PyCall%20and%20PythonCall.ipynb) using Python from Julia with PyCall**
- [special extra](https://mybinder.org/v2/gh/jolin-io/workshop-accelerate-Python-with-Julia/main?filepath=03-example-cython-vs-cpp-vs-julia.ipynb) - Julia vs C++

In [ ]:
# small helper for html interaction, export @htl macro
using HypertextLiteral

In [ ]:
# show plotly plot
show_plotly(fig, width="80%", height="500px") = mktempdir() do tmp 
    fig.write_html("$tmp/result.html")
    @htl """<iframe width="$width" height="$height" srcdoc="$(readchomp("$tmp/result.html"))"/>"""
end

# PyCall

We reuse binder's default Python (and also support a local virtual env for developing this notebook locally in vscode).

In [ ]:
local_venv = abspath(joinpath(Base.current_project(), "..", ".venv", "bin", "python3"))
ENV["PYTHON"] = isfile(local_venv) ? local_venv : "python3"
import PyCall
import PyCall: @py_str

In [ ]:
pd = PyCall.pyimport("pandas")
pd.options.plotting.backend = "plotly"

In [ ]:
n = 4
pdf = pd.DataFrame(Dict(:a => 1:n, :b => rand(n), :c => sin.(1:n)))

In [ ]:
fig = pdf.plot(kind="bar", x="a", y="c")
show_plotly(fig)

### What to do if you just want to execute some python?

In [ ]:
py"""
import numpy as np
# use $() to use variables from the running julia process
$pdf["sin"] = np.sin($pdf.a)
"""

NOTE that the multiline `py"""..."""` string does not have a return value.

For return values you need to use it singleline `py"..."`

In [ ]:
py"$pdf[$pdf.a > 2]"

In [ ]:
show_plotly(pdf.plot(x="a", y="sin"))

# PythonCall

PythonCall does not have `py"..."` string, but comes with a `@py` macro.

In [ ]:
# we can reuse the same Python as PyCall uses
ENV["JULIA_PYTHONCALL_EXE"] = "@PyCall"
import PythonCall
import PythonCall: @py

While you could write
```julia
pd2 = PythonCall.pyimport("pandas")
```
just like in PyCall, you can also do

In [ ]:
@py import pandas as pd2
pd2.options.plotting.backend = "matplotlib"

Because PythonCall does not autoconvert anything, we need a bit more manual work here and there.

In our example of constructing a pandas DataFrame from a julia Dict we need to:
- use Strings as keys
- use explicit transformation to Python's `dict` type

As you have seen previously, the support for transforming julia DataFrame to pandas DataFrame is better actually.

In [ ]:
n = 4
pdf2 = pd2.DataFrame(PythonCall.pybuiltins.dict(
    Dict("a" => 1:n, "b" => rand(n), "c" => sin.(1:n))
))

PythonCall comes with pretty printing support for matplotlib figures

In [ ]:
pdf2.plot(kind="bar", x="a", y="b").figure

In [ ]:
@py pdf2[pdf2.a > 2]
# This actually also works without @py.
# Sometimes the julia interpretation is not enough and may lead to errors.
# Then using @py will probably solve the issues for you.

### Some further notes on conversions

This is specific to PythonCall. (PyCall does auto-conversion for you.)

- `PythonCall.Py(...)` transform julia value to python value (no conversion, just wrapping).
  - Everything is automatically wrapped by `Py` if you use python functions.
  - also inside `@py` everything is automatically wrapped
  - if you want to convert some julia value to a python value, then use the python constructors on the python wrappers
- `pyconvert(Type, pythonvalue)` transform python value to julia value (with conversion)


In [ ]:
PythonCall.Py(" ").join(["hello", "world"])

or alternatively with `@py`

In [ ]:
sentence = @py " ".join(["hello", "world"])

In [ ]:
# get a real Julia String
PythonCall.pyconvert(String, sentence)

### Something like py"..."?

In case you just have some plain python which you want to run, use `@pyexec` and `@pyeval`.

> Note that `@pyexec`/ `@pyeval` do not support interpolation

In [ ]:
PythonCall.@pyexec """
# You need global pyfun because @pyexec introduces a new local scope.
global pyfun
def pyfun(a):
    return 2*a + 9
"""

In [ ]:
pyfun = PythonCall.@pyeval "pyfun"

# Next: [special extra, comparing C++ & Julia](https://mybinder.org/v2/gh/jolin-io/workshop-accelerate-Python-with-Julia/main?filepath=03-example-cython-vs-cpp-vs-julia.ipynb)

For questions or suggestions please contact me at stephan.sahm@jolin.io

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>